# Scrape Data From Reddit Pushshift

In [1]:
from Reddit_Loader import Loader
import datetime
import pandas as pd
import pickle


In [12]:
# Set Start and End Date
START = "01/01/2020"
END = datetime.datetime.now().strftime("%d/%m/%Y") #today

In [13]:
# Load data from Pushshift and store it as a dataframe
l = Loader("gme", START, END, "wallstreetbets")
submissions = l.load_submissions()

Response cache key: 29983c8c40dd541667743b8ea5bd1315


Not all PushShift shards are active. Query results may be incomplete.


179167 result(s) available in Pushshift


Not all PushShift shards are active. Query results may be incomplete.


Total:: Success Rate: 80.71% - Requests: 2649 - Batches: 266 - Items Remaining: 354


Not all PushShift shards are active. Query results may be incomplete.


354 result(s) not found in Pushshift
Retrieved 178813 submissions from Pushshift.


In [15]:
# Save as csv
submissions.to_csv('./data/submissions.csv', encoding='utf-8', index=False)

In [14]:
# Save dataframe as pickle
submissions.to_pickle("./data/submissions.pkl")

In [3]:
submissions = pd.read_pickle("./data/submissions.pkl")

# Data Exploration

In [92]:
# All the available columns in the submissions dataframe
for element in submissions:
    print(element, end = ", ")

all_awardings, allow_live_comments, author, author_flair_css_class, author_flair_richtext, author_flair_text, author_flair_type, author_fullname, author_patreon_flair, author_premium, awarders, can_mod_post, contest_mode, created_utc, domain, full_link, gildings, id, is_crosspostable, is_meta, is_original_content, is_reddit_media_domain, is_robot_indexable, is_self, is_video, link_flair_background_color, link_flair_css_class, link_flair_richtext, link_flair_template_id, link_flair_text, link_flair_text_color, link_flair_type, locked, media_only, no_follow, num_comments, num_crossposts, over_18, parent_whitelist_status, permalink, pinned, pwls, retrieved_on, score, selftext, send_replies, spoiler, stickied, subreddit, subreddit_id, subreddit_subscribers, subreddit_type, suggested_sort, thumbnail, title, total_awards_received, treatment_tags, upvote_ratio, url, whitelist_status, wls, post_hint, preview, removed_by_category, is_gallery, thumbnail_height, thumbnail_width, url_overridden_by

In [78]:
# The number of Awards can be used as a proxy for the awards given
print(len(submissions["all_awardings"][44][0]["resized_icons"])) # We only count the unique awards given, not if an award has been given multiple times. May figure it out later.

# is_video can be useful to weed out posts, that do not have any content and only a video. Might still be able to perform sentiment analysis on the title.
# link_flair_text can be useful to classify content: Meme, Yolo, etc.

5


In [220]:
print(type(submissions["num_comments"][76]))
submissions["num_comments"][76]

<class 'numpy.int64'>


0

In [216]:
for i, elem in enumerate(submissions["no_follow"]):
    if elem != np.bool(False):
        print(i)

NameError: name 'np' is not defined

In [107]:
# Drop unneeded columns
## allow_live_comments is Boolean
## author_flair_css_class shows the css class of the author_flair
## author_flair_richtext, author_flair_text, author_flair_type can be ignored, because the author can assign a flair to himself if he wants to
## author_fullname is just the id for the user
## author_patreon_flair, author_premium, domain, gildings, is_crosspostable, is_original_content, is_reddit_media_domain, is_robot_indexable,
## is_self, link_flair_background_color, link_flair_text_color, link_flair_css_class, link_flair_template_id, locked, media_only,
## no_follow do not seem to be helpful for sentiment analysis
## awarders is always empty
## can_mod_post, contest_mode, is_meta are always False
## link_flair_type doesn't really say anything
## link_flair_richtext is a dictionary for for link_flair(which is not discarded)
submissions.drop(columns = ["allow_live_comments", "author_flair_css_class", "author_flair_richtext", 
                            "author_flair_text", "author_flair_type", "author_fullname", "author_patreon_flair",
                            "author_premium", "domain", "awarders", "can_mod_post", "contest_mode",
                            "gildings", "is_crosspostable", "is_meta", "is_original_content", "is_reddit_media_domain",
                            "is_robot_indexable", "is_self", "link_flair_background_color", "link_flair_text_color",
                            "link_flair_type", "link_flair_css_class", "link_flair_richtext", "link_flair_template_id",
                            "locked", "media_only", "no_follow"])

,all_awardings,author,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,awarders,can_mod_post,...,discussion_type,crosspost_parent,crosspost_parent_list,author_flair_template_id,author_is_blocked,is_created_from_ads_ui,steward_reports,gilded,distinguished,collections
0,[],guitarheropwn21,[],None,text,t2_9g956,False,False,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],stonkylassie,[],None,text,t2_80q8h8pv,False,False,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],Bridge4YC,[],None,text,t2_3g7567fg,False,False,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],ChingityChingtyChong,[],None,text,t2_2dbxwncy,False,False,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],Poopoosexman,[],None,text,t2_50tnyl7m,False,False,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178808,[],robbe-cl,[],None,text,t2_7b1xle97,False,False,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178809,[],powleads,[],None,text,t2_8cii0hir,False,False,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178810,[],Liftedlarvitar,[],None,text,t2_16n5qi,False,False,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178811,[],Fortnox-_-,[],None,text,t2_a0rqwcp8,False,False,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
